In [ ]:
import cv2
import os
import threading
from ultralytics import YOLO
from playsound import playsound  # Bibliothek für Sound-Wiedergabe

# Klassen aus der Datei classes.txt laden
with open("classes.txt", "r") as f:
    class_names = [line.strip() for line in f.readlines()]

# Zuordnung von Klassen zu ihren Sounds
sounds = {
    "Nasebohren": "nasebohren.mp3",
    "Nägelkauen": "naegelkauen.mp3",
}

# Sicherstellen, dass alle Dateien existieren
for class_name, sound_file in sounds.items():
    if not os.path.exists(sound_file):
        raise FileNotFoundError(f"Sound file for {class_name} not found: {sound_file}")

# YOLOv8-Modell laden
model = YOLO("best.pt")

# Kamera öffnen
cap = None
for camera_index in range(5):  # Versuche Kamera-Indices von 0 bis 4
    cap = cv2.VideoCapture(camera_index)
    if cap.isOpened():
        print(f"Kamera erfolgreich geöffnet mit Index {camera_index}.")
        break
    else:
        cap.release()
        cap = None

if cap is None or not cap.isOpened():
    raise RuntimeError("Keine Kamera konnte geöffnet werden. Stellen Sie sicher, dass eine Kamera verfügbar ist und nicht von einer anderen Anwendung verwendet wird.")

print("Drücken Sie 'q', um das Programm zu beenden.")

# Flag für Sound-Wiedergabe
sound_playing = threading.Event()

def play_sound(sound_file):
    """Spielt den angegebenen Sound ab und blockiert währenddessen weitere Erkennung."""
    sound_playing.set()
    playsound(sound_file)
    sound_playing.clear()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fehler beim Lesen des Kamera-Feeds.")
        break

    # Überspringe neue Frames, wenn Sound abgespielt wird
    if sound_playing.is_set():
        continue

    # YOLO-Vorhersagen
    results = model.predict(source=frame, show=False, conf=0.5)

    # Ergebnisse durchlaufen
    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])  # Klassenindex
            class_name = class_names[cls] if cls < len(class_names) else "Unbekannt"

            # Sound abspielen, wenn Klasse erkannt wird
            if class_name in sounds and not sound_playing.is_set():
                print(f"Erkannt: {class_name}")
                threading.Thread(target=play_sound, args=(sounds[class_name],), daemon=True).start()

            # Bounding Box zeichnen
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            label = f"{class_name} {confidence:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Frame anzeigen
    cv2.imshow("YOLOv8 Detection", frame)

    # Beenden, wenn 'q' gedrückt wird
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
